In [4]:
import os

In [5]:
import tensorflow as tf

In [6]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [7]:
from keras.applications.vgg16 import VGG16 as VGG

In [8]:
from keras.applications.densenet import DenseNet201 as DenseNet

In [9]:
from keras.optimizers import SGD

In [10]:
from keras.layers import GlobalAveragePooling2D, Dense

In [11]:
from keras.models import Model

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [13]:
def preprocessing_image_rgb(x):
    # define mean and std values
    mean = [87.845, 96.965, 103.947]
    std = [23.657, 16.474, 13.793]
    # loop over image channels
    for idx, mean_value in enumerate(mean):
        x[..., idx] -= mean_value
        x[..., idx] /= std[idx]
    return x


def preprocessing_image_ms(x):
    # define mean and std values
    mean = [1353.036, 1116.468, 1041.475, 945.344, 1198.498, 2004.878,
            2376.699, 2303.738, 732.957, 12.092, 1818.820, 1116.271, 2602.579]
    std = [65.479, 154.008, 187.997, 278.508, 228.122, 356.598, 456.035,
           531.570, 98.947, 1.188, 378.993, 303.851, 503.181]
    # loop over image channels
    for idx, mean_value in enumerate(mean):
        x[..., idx] -= mean_value
        x[..., idx] /= std[idx]
    return x


def categorical_label_from_full_file_name(files, class_indices):
    from keras.utils import to_categorical
    import os
    # file basename without extension
    base_name = [os.path.splitext(os.path.basename(i))[0] for i in files]
    # class label from filename
    base_name = [i.split("_")[0] for i in base_name]
    # label to indices
    image_class = [class_indices[i] for i in base_name]
    # class indices to one-hot-label
    return to_categorical(image_class, num_classes=len(class_indices))


def simple_image_generator(files, class_indices, batch_size=32,
                           rotation_range=0, horizontal_flip=False,
                           vertical_flip=False):
    from skimage.io import imread
    from skimage.transform import rotate
    import numpy as np
    from random import sample, choice

    while True:
        # select batch_size number of samples without replacement
        batch_files = sample(files, batch_size)
        # get one_hot_label
        batch_Y = categorical_label_from_full_file_name(batch_files,
                                                        class_indices)
        # array for images
        batch_X = []
        # loop over images of the current batch
        for idx, input_path in enumerate(batch_files):
            image = np.array(imread(input_path), dtype=float)
            image = preprocessing_image_ms(image)
            # process image
            if horizontal_flip:
                # randomly flip image up/down
                if choice([True, False]):
                    image = np.flipud(image)
            if vertical_flip:
                # randomly flip image left/right
                if choice([True, False]):
                    image = np.fliplr(image)
            # rotate image by random angle between
            # -rotation_range <= angle < rotation_range
            if rotation_range is not 0:
                angle = np.random.uniform(low=-abs(rotation_range),
                                          high=abs(rotation_range))
                image = rotate(image, angle, mode='reflect',
                               order=1, preserve_range=True)
            # put all together
            batch_X += [image]
        # convert lists to np.array
        X = np.array(batch_X)
        Y = np.array(batch_Y)

        yield(X, Y)


In [14]:
path_to_split_datasets = "~/RGB"

In [15]:
use_vgg = True

In [16]:
batch_size = 64

In [17]:
path_to_home = os.path.expanduser("E:/PAPER_RS_DEEPLEARNING/image")

In [18]:
path_to_split_datasets = path_to_split_datasets.replace("~", path_to_home)

In [19]:
path_to_split_datasets

'E:/PAPER_RS_DEEPLEARNING/image/RGB'

In [20]:
path_to_train = os.path.join(path_to_split_datasets, "train")

In [21]:
path_to_train

'E:/PAPER_RS_DEEPLEARNING/image/RGB\\train'

In [22]:
path_to_validation = os.path.join(path_to_split_datasets, "validation")

In [23]:
path_to_validation

'E:/PAPER_RS_DEEPLEARNING/image/RGB\\validation'

In [24]:
sub_dirs = [sub_dir for sub_dir in os.listdir(path_to_train)
            if os.path.isdir(os.path.join(path_to_train, sub_dir))]

In [25]:
num_classes=(len(sub_dirs))

In [26]:
if use_vgg:
    base_model = VGG(include_top = False,
                     weights = 'imagenet',
                    input_shape=(64,64,3))
else:
    base_model = DenseNet(include_top = False,
                          weights = 'imagenet',
                          input_shape=(64,64,3))

In [27]:
top_model = base_model.output

In [28]:
top_model = GlobalAveragePooling2D()(top_model)

In [29]:
top_model

<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(?, 512) dtype=float32>

In [30]:
if use_vgg:
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)

In [31]:
predictions = Dense(num_classes, activation='softmax')(top_model)

In [32]:
model = Model(inputs=base_model.input, outputs=predictions)

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [34]:
train_datagen = ImageDataGenerator(fill_mode="reflect",
                                    rotation_range=45,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    preprocessing_function=preprocessing_image_rgb)

In [35]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_image_rgb)

In [36]:
train_generator = train_datagen.flow_from_directory(path_to_train,
                                                    target_size=(64,64),
                                                    batch_size=batch_size,
                                                    class_mode = 'categorical')

Found 18000 images belonging to 45 classes.


In [37]:
class_indices = train_generator.class_indices

In [38]:
print(class_indices)

{'overpass': 26, 'cloud': 9, 'meadow': 22, 'mountain': 25, 'stadium': 39, 'palace': 27, 'desert': 12, 'airport': 1, 'runway': 34, 'sparse_residential': 38, 'rectangular_farmland': 31, 'golf_course': 15, 'airplane': 0, 'railway': 29, 'river': 32, 'parking_lot': 28, 'harbor': 17, 'terrace': 42, 'snowberg': 37, 'chaparral': 6, 'roundabout': 33, 'intersection': 19, 'forest': 13, 'railway_station': 30, 'basketball_court': 3, 'storage_tank': 40, 'freeway': 14, 'ship': 36, 'wetland': 44, 'dense_residential': 11, 'baseball_diamond': 2, 'lake': 21, 'thermal_power_station': 43, 'sea_ice': 35, 'medium_residential': 23, 'bridge': 5, 'commercial_area': 10, 'circular_farmland': 8, 'island': 20, 'industrial_area': 18, 'ground_track_field': 16, 'tennis_court': 41, 'beach': 4, 'church': 7, 'mobile_home_park': 24}


In [39]:
validation_generator = test_datagen.flow_from_directory(path_to_validation, 
                                                        target_size=(64, 64), 
                                                        batch_size=batch_size, 
                                                        class_mode='categorical') 


Found 13500 images belonging to 45 classes.


In [40]:
for layer in base_model.layers:
    layer.trainable = False

In [41]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [42]:
if use_vgg:
    file_name = "vgg"

else:
    file_name = "dense"

In [43]:
checkpointer = ModelCheckpoint("E:/PAPER_RS_DEEPLEARNING/image/RGB/" + file_name + 
                               "_rgb_transfer_init." + 
                              "{epoch:02d}-{val_categorical_accuracy:.3f}." +
                               "hdf5",
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True,
                               mode='max')

In [44]:
earlystopper = EarlyStopping(monitor='val_categorical_accuracy', 
                             patience=10, 
                             mode='max', 
                             restore_best_weights=True) 

In [45]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_grads=True, 
                          write_images=True, update_freq='epoch') 


In [48]:
history = model.fit_generator(train_generator, 
                               steps_per_epoch=200, 
                               epochs=1, 
                               callbacks=[checkpointer, earlystopper, 
                                          tensorboard], 
                               validation_data=validation_generator, 
                               validation_steps=500) 

Epoch 1/1
200/200 [==============================] - 119s 595ms/step - loss: 1.4756 - categorical_accuracy: 0.5675 - val_loss: 1.4065 - val_categorical_accuracy: 0.5859

Epoch 00001: val_categorical_accuracy improved from 0.46449 to 0.58590, saving model to E:/PAPER_RS_DEEPLEARNING/image/RGB/vgg_rgb_transfer_init.01-0.586.hdf5


In [49]:
initial_epoch = len(history.history['loss'])+1

In [50]:
names = []

In [51]:
for i, layer in enumerate(model.layers): 
    names.append([i, layer.name, layer.trainable]) 

In [52]:
print(names)

[[0, 'input_1', False], [1, 'block1_conv1', False], [2, 'block1_conv2', False], [3, 'block1_pool', False], [4, 'block2_conv1', False], [5, 'block2_conv2', False], [6, 'block2_pool', False], [7, 'block3_conv1', False], [8, 'block3_conv2', False], [9, 'block3_conv3', False], [10, 'block3_pool', False], [11, 'block4_conv1', False], [12, 'block4_conv2', False], [13, 'block4_conv3', False], [14, 'block4_pool', False], [15, 'block5_conv1', False], [16, 'block5_conv2', False], [17, 'block5_conv3', False], [18, 'block5_pool', False], [19, 'global_average_pooling2d_1', True], [20, 'dense_1', True], [21, 'dense_2', True], [22, 'dense_3', True]]


In [53]:
if use_vgg: 
     for layer in model.layers[:4]: 
            layer.trainable = False 
     for layer in model.layers[4:]: 
         layer.trainable = True 
else: 
     for layer in model.layers[:7]: 
             layer.trainable = False 
     for layer in model.layers[7:]: 
             layer.trainable = True 


In [54]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
               loss='categorical_crossentropy', 
               metrics=['categorical_accuracy']) 

In [55]:
if use_vgg: 
     file_name = "vgg" 
else: 
     file_name = "dense" 


In [56]:
checkpointer = ModelCheckpoint("E:/PAPER_RS_DEEPLEARNING/image/RGB/" + file_name + 
                                "_rgb_transfer_final." + 
                                "{epoch:02d}-{val_categorical_accuracy:.3f}" + 
                                ".hdf5", 
                                monitor='val_categorical_accuracy', 
                                verbose=1, 
                                save_best_only=True, 
                                mode='max') 

In [57]:
earlystopper = EarlyStopping(monitor='val_categorical_accuracy', 
                              patience=50, 
                              mode='max') 

In [58]:
model.fit_generator(train_generator, 
                    steps_per_epoch=100, 
                    epochs=10, 
                    callbacks=[checkpointer, earlystopper, tensorboard], 
                    validation_data=validation_generator, 
                    validation_steps=500, 
                    initial_epoch=initial_epoch) 


Epoch 3/10
100/100 [==============================] - 66s 664ms/step - loss: 1.1378 - categorical_accuracy: 0.6608 - val_loss: 1.2403 - val_categorical_accuracy: 0.6353

Epoch 00003: val_categorical_accuracy improved from -inf to 0.63525, saving model to E:/PAPER_RS_DEEPLEARNING/image/RGB/vgg_rgb_transfer_final.03-0.635.hdf5
Epoch 4/10
100/100 [==============================] - 66s 658ms/step - loss: 1.0071 - categorical_accuracy: 0.6927 - val_loss: 1.1167 - val_categorical_accuracy: 0.6709

Epoch 00004: val_categorical_accuracy improved from 0.63525 to 0.67092, saving model to E:/PAPER_RS_DEEPLEARNING/image/RGB/vgg_rgb_transfer_final.04-0.671.hdf5
Epoch 5/10
100/100 [==============================] - 68s 679ms/step - loss: 0.9570 - categorical_accuracy: 0.7119 - val_loss: 1.1598 - val_categorical_accuracy: 0.6596

Epoch 00005: val_categorical_accuracy did not improve from 0.67092
Epoch 6/10
100/100 [==============================] - 63s 631ms/step - loss: 0.8887 - categorical_accuracy